In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [8]:
!pip install tensorflow

train

In [ ]:
!python "/content/drive/MyDrive/城市工程系统智能化/MobileViT/train.py"

2023-12-05 15:36:05.444941: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 15:36:05.445011: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 15:36:05.445052: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 15:36:05.455165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 15:36:06.707755: W tensorflow/compiler/

test

In [ ]:
!python '/content/drive/MyDrive/城市工程系统智能化/MobileViT/test.py'

attention assisted augmentation training
基于注意力机制的数据增强策略

对每个子文件夹按顺序抽取30%进行训练，然后在测试集上进行测试，或者也抽取30%进行测试。分别得到该模型在各个类上的损失（多尝试几种），对损失求softmax，根据得到的值对数据进行数据增强（从增强数据文件夹中抽取%数量的数据进行训练），值越大数据增强得到的数据越多。
对每个子文件夹抽取30%的数据进行训练，对剩下的数据进行数据增强，对剩下的数据加上增强的数据取一半再次进行训练，然后对剩下的一半数据增强。

1、将一个文件夹中各个子文件夹中的图片划分为90%的训练集和10%的测试集。其中，每个子文件夹中是不同种类的数据，文件名是对应的种类名
2、抽取各个子文件夹的训练集中33%的数据进行训练
3、在各个子文件夹测试集上进行测试，得到损失，求softmax=?%
4、在各个子文件夹的增强数据集中取出?%×30%的数据和剩下训练集的一半进行训练
5、在各个子文件夹测试集上进行测试，得到损失，求softmax=?%
6、在各个子文件夹剩余的增强数据集中取出?%×30%的数据和剩下的训练集进行训练


In [ ]:
import os
import shutil
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# 设置随机种子以保证结果的可重复性
random.seed(42)
torch.manual_seed(42)

# 定义LSTM模型（示例模型）
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

# 数据集路径
# 创建文件夹
for i in range(1, 4):
    split_train_folder = f"train_split_{i}"
    os.makedirs(split_train_folder, exist_ok=True)

os.makedirs("test", exist_ok=True)

dataset_path = "/content/drive/MyDrive/城市工程系统智能化/Soil types"
num_classes = 5

# 划分训练集和测试集
def split_dataset(dataset_path, train_ratio=0.8):
    classes = os.listdir(dataset_path)
    train_dir = "train"
    test_dir = "test"

    for cls in classes:
        cls_path = os.path.join(dataset_path, cls)
        images = os.listdir(cls_path)
        random.shuffle(images)

        train_size = int(len(images) * train_ratio)
        train_images = images[:train_size]
        test_images = images[train_size:]

        # 创建训练集和测试集文件夹
        train_cls_path = os.path.join(train_dir, cls)
        test_cls_path = os.path.join(test_dir, cls)

        os.makedirs(train_cls_path, exist_ok=True)
        os.makedirs(test_cls_path, exist_ok=True)

        # 将图像拷贝到相应文件夹
        for img in train_images:
            shutil.copy(os.path.join(cls_path, img), os.path.join(train_cls_path, img))
        for img in test_images:
            shutil.copy(os.path.join(cls_path, img), os.path.join(test_cls_path, img))

# 划分训练集为三个子集
def split_train_dataset(train_path, ratios=[0.4, 0.3, 0.3]):
    classes = os.listdir(train_path)

    for cls in classes:
        cls_path = os.path.join(train_path, cls)
        images = os.listdir(cls_path)
        random.shuffle(images)

        sizes = [int(ratio * len(images)) for ratio in ratios]
        splits = [images[:sizes[0]], images[sizes[0]:sizes[0] + sizes[1]], images[sizes[0] + sizes[1]:]]

        # 创建三个子集文件夹
        for i, split in enumerate(splits):
            split_cls_path = os.path.join(f"train_split_{i + 1}", cls)
            os.makedirs(split_cls_path, exist_ok=True)

            # 将图像拷贝到相应文件夹
            for img in split:
                shutil.copy(os.path.join(cls_path, img), os.path.join(split_cls_path, img))

# 定义数据增强和转换
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

# 设置超参数
input_size = 224
hidden_size = 128
num_layers = 2
learning_rate = 0.001
epochs = 10

# 初始化LSTM模型和损失函数
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(epochs):
    # 第一个拆分的训练集
    split_train_dataset("train_split_1")

    # 数据加载
    train_dataset = ImageFolder("train_split_1", transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for batch in train_loader:
        inputs, labels = batch

        # 模型训练
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 测试集测试
    test_dataset = ImageFolder("test", transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    class_losses = {i: 0.0 for i in range(num_classes)}
    class_counts = {i: 0 for i in range(num_classes)}

    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            for i in range(num_classes):
                mask = labels == i
                class_losses[i] += torch.sum(loss[mask]).item()
                class_counts[i] += mask.sum().item()

    # 计算各个类别的损失值
    for i in range(num_classes):
        class_losses[i] /= class_counts[i]

    # 求softmax得到各个类别的增强比
    exp_losses = torch.exp(torch.tensor(list(class_losses.values())))
    enhancement_ratios = exp_losses / exp_losses.sum()

    # 对第二个划分的训练集进行数据增强
    split_train_dataset("train_split_2", ratios=enhancement_ratios.tolist())

    # 数据加载
    train_dataset = ImageFolder("train_split_2", transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for batch in train_loader:
        inputs, labels = batch

        # 模型训练
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 对第三个训练集进行数据增强
    split_train_dataset("train_split_3", ratios=enhancement_ratios.tolist())

    # 数据加载
    train_dataset = ImageFolder("train_split_3", transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for batch in train_loader:
        inputs, labels = batch

        # 模型训练
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 使用全部测试集测试
    test_dataset = ImageFolder("test", transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    class_losses = {i: 0.0 for i in range(num_classes)}
    class_counts = {i: 0 for i in range(num_classes)}

    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            for i in range(num_classes):
                mask = labels == i
                class_losses[i] += torch.sum(loss[mask]).item()
                class_counts[i] += mask.sum().item()

    # 计算各个类别的损失值
    for i in range(num_classes):
        class_losses[i] /= class_counts[i]

    print(f"Epoch {epoch + 1}/{epochs} completed")

# 训练结束后保存模型
torch.save(model.state_dict(), "lstm_model.pth")


FileNotFoundError: ignored

In [15]:
#这段代码得到的数据集不正宗
import os
import shutil
import random

def split_dataset(dataset_path, save_path, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # 获取所有类别文件夹
    classes = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]

    for class_folder in classes:
        class_path = os.path.join(dataset_path, class_folder)
        images = [img for img in os.listdir(class_path) if img.endswith('.jpg') or img.endswith('.png')]
        random.shuffle(images)

        total_images = len(images)
        train_size = int(total_images * train_ratio)
        val_size = int(total_images * val_ratio)
        test_size = total_images - train_size - val_size

        # 创建保存路径
        save_train_path = os.path.join(save_path, 'train', class_folder)
        save_val_path = os.path.join(save_path, 'val', class_folder)
        save_test_path = os.path.join(save_path, 'test', class_folder)

        os.makedirs(save_train_path, exist_ok=True)
        os.makedirs(save_val_path, exist_ok=True)
        os.makedirs(save_test_path, exist_ok=True)

        # 划分训练集
        train_set = images[:train_size]
        for img in train_set:
            shutil.copy(os.path.join(class_path, img), os.path.join(save_train_path, img))
        # 分别创建三个子文件夹
        #train_set1_path = os.path.join(save_train_path, 'train_set1')
        #train_set2_path = os.path.join(save_train_path, 'train_set2')
        #train_set3_path = os.path.join(save_train_path, 'train_set3')

        #os.makedirs(train_set1_path, exist_ok=True)
        #os.makedirs(train_set2_path, exist_ok=True)
        #os.makedirs(train_set3_path, exist_ok=True)

        # 将数据均匀分配到三个子文件夹中
        #for i, img in enumerate(train_set):
         #   if i % 3 == 0:
          #      shutil.copy(os.path.join(class_path, img), os.path.join(train_set1_path, img))
           # elif i % 3 == 1:
            #    shutil.copy(os.path.join(class_path, img), os.path.join(train_set2_path, img))
            #else:
             #   shutil.copy(os.path.join(class_path, img), os.path.join(train_set3_path, img))

        # 划分验证集
        val_set = images[train_size:train_size + val_size]
        for img in val_set:
            shutil.copy(os.path.join(class_path, img), os.path.join(save_val_path, img))

        # 划分测试集
        test_set = images[train_size + val_size:]
        for img in test_set:
            shutil.copy(os.path.join(class_path, img), os.path.join(save_test_path, img))

# 用法示例
dataset_path = '/content/drive/MyDrive/城市工程系统智能化/Soil types'
save_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented'
split_dataset(dataset_path, save_path)


In [ ]:
##这个代码得到的attention_augmented1是最正规的数据集
import os
import shutil
import random

def split_dataset(dataset_path, save_path, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # 获取所有类别文件夹
    classes = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]

    for class_folder in classes:
        class_path = os.path.join(dataset_path, class_folder)
        images = [img for img in os.listdir(class_path) if img.endswith('.jpg') or img.endswith('.png')]
        random.shuffle(images)

        total_images = len(images)
        train_size = int(total_images * train_ratio)
        val_size = int(total_images * val_ratio)
        test_size = total_images - train_size - val_size

        # 创建保存路径
        save_train_path = os.path.join(save_path, 'train')
        save_val_path = os.path.join(save_path, 'val', class_folder)
        save_test_path = os.path.join(save_path, 'test', class_folder)

        os.makedirs(save_train_path, exist_ok=True)
        os.makedirs(save_val_path, exist_ok=True)
        os.makedirs(save_test_path, exist_ok=True)

        # 划分训练集
        train_set = images[:train_size]

        # 分别创建三个子文件夹
        train_set1_path = os.path.join(save_train_path, 'train_set1', class_folder)
        train_set2_path = os.path.join(save_train_path, 'train_set2', class_folder)
        train_set3_path = os.path.join(save_train_path, 'train_set3', class_folder)

        os.makedirs(train_set1_path, exist_ok=True)
        os.makedirs(train_set2_path, exist_ok=True)
        os.makedirs(train_set3_path, exist_ok=True)

        # 将数据均匀分配到三个子文件夹中
        for i, img in enumerate(train_set):
            if i % 3 == 0:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_set1_path, img))
            elif i % 3 == 1:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_set2_path, img))
            else:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_set3_path, img))

        # 划分验证集
        val_set = images[train_size:train_size + val_size]
        for img in val_set:
            shutil.copy(os.path.join(class_path, img), os.path.join(save_val_path, img))

        # 划分测试集
        test_set = images[train_size + val_size:]
        for img in test_set:
            shutil.copy(os.path.join(class_path, img), os.path.join(save_test_path, img))

# 用法示例
dataset_path = '/content/drive/MyDrive/城市工程系统智能化/Soil types'
save_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented1'
split_dataset(dataset_path, save_path)


In [63]:
#没有进行数据增强的训练

'''
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.classes = os.listdir(folder_path)
        self.samples = []
        for class_name in self.classes:
            class_path = os.path.join(folder_path, class_name)
            for sample_set in os.listdir(class_path):
                sample_set_path = os.path.join(class_path, sample_set)
                self.samples.extend([(os.path.join(sample_set_path, filename), int(class_name)) for filename in os.listdir(sample_set_path)])

        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# LSTM模型定义
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1, :, :])
        return out

# 训练模型
def train_model(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(train_loader)

# 测试模型
def test_model(test_loader, model, device):
    model.eval()
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_outputs)
    return accuracy

# 数据增强函数
def data_augmentation(data_path, enhancement_ratios):
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
    ])

    enhanced_dataset = CustomDataset(data_path, transform=transform)
    return enhanced_dataset

# 主训练循环
def train_loop(train_paths, val_path, test_path, num_epochs=10):
    input_size = 224
    hidden_size = 128
    num_classes = 5
    learning_rate = 0.001
    batch_size = 16

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = LSTMModel(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        # 数据增强函数
        enhancement_ratios = None
        if epoch > 0:  # 不是第一个epoch才进行数据增强
            # 对第一个拆分的子训练集进行测试，获取各个类别的损失值
            train_loss_set1 = test_model(train_paths[0], model, criterion, device)

            # 随机选取30%的图像数据进行测试
            sampled_indices = random.sample(range(len(train_loss_set1)), int(0.3 * len(train_loss_set1)))
            sampled_loss_set1 = [train_loss_set1[i] for i in sampled_indices]

            # 对损失值求softmax得到各个类别的增强比
            softmax_probs = nn.functional.softmax(torch.tensor(sampled_loss_set1), dim=0)
            enhancement_ratios = softmax_probs.numpy()

        for train_path in train_paths:
            # 对当前子训练集进行数据增强
            enhanced_train_set = data_augmentation(train_path, enhancement_ratios)

            # 对增强后的子训练集进行模型训练
            enhanced_train_loader = DataLoader(enhanced_train_set, batch_size=batch_size, shuffle=True, num_workers=4)
            train_model(enhanced_train_loader, model, criterion, optimizer, device)

        # 训练模型
        train_loss = test_model(train_paths[0], model, criterion, device)

        # 在验证集上测试模型
        val_accuracy = test_model(val_path, model, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # 在测试集上测试模型
        test_accuracy = test_model(test_path, model, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {test_accuracy:.4f}")

# 数据集文件夹路径
base_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented'
train_paths = [os.path.join(base_path, 'train', class_name) for class_name in os.listdir(os.path.join(base_path, 'train'))]
val_path = os.path.join(base_path, 'val')
test_path = os.path.join(base_path, 'test')

# 运行主训练循环
train_loop(train_paths, val_path, test_path, num_epochs=10)
'''
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.functional import softmax
import random
import torch.nn.functional as F
import os

# 设定种子以确保可重复性
torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Reshape input to (batch_size, sequence_length, input_size)
        x = x.view(x.size(0), x.size(3), -1).permute(0, 2, 1)

        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

# 数据增强
def data_augmentation(data, enhancement_ratios):
    # 这里只是一个示例，你需要根据你的需求进行适当的数据增强操作
    augmented_data = data * enhancement_ratios.unsqueeze(0).unsqueeze(-1).expand_as(data)
    return augmented_data

# 训练模型
def train(model, train_loader, criterion, optimizer, epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试模型
def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}\n')

    return test_loss, accuracy

# 数据集路径
dataset_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented'
#不进行数据增强的数据集路径
#dataset_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented'
# 定义转换
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor()])

# 创建数据集和数据加载器
#train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path,'train', 'train_set1'), transform=transform)
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path,'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path,'val'), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'test'), transform=transform)

# 划分验证集
#validation_split = 0.1
#dataset_size = len(train_dataset)
#indices = list(range(dataset_size))
#split = int(validation_split * dataset_size)

#train_indices, val_indices = indices[split:], indices[:split]

#train_sampler = SubsetRandomSampler(train_indices)
#val_sampler = SubsetRandomSampler(val_indices)

# 数据加载器
#train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler)
#val_loader = DataLoader(train_dataset, batch_size=64, sampler=val_sampler)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
input_size = 32  # 输入的大小，根据你的数据集调整
hidden_size = 64  # LSTM隐藏层的大小，根据需要调整
num_classes = 5  # 类别数，根据你的数据集调整
model = LSTMModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

test_dataset_path = os.path.join(dataset_path, 'test')
final_test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)
final_test_loader = DataLoader(final_test_dataset, batch_size=64, shuffle=True)
# 训练10个epoch
'''
for epoch in range(10):
    train(model, train_loader, criterion, optimizer, epoch, device)

    # 在整个测试集上进行最终测试
    final_test_loss, final_test_accuracy = test(model, final_test_loader, device)

    print(f'Final Test Accuracy: {final_test_accuracy:.2f}')
'''
for epoch in range(200):
    train(model, train_loader, criterion, optimizer, epoch, device)

    # 在验证集上进行测试
    val_loss, val_accuracy = test(model, val_loader, device)
    print(f'Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}')

# 在整个测试集上进行最终测试
final_test_loss, final_test_accuracy = test(model, test_loader, device)
print(f'Final Test Accuracy: {final_test_accuracy:.2f}')
'''
for epoch in range(10):
    train(model, train_loader, criterion, optimizer, epoch, device)

    # 在测试集每个子文件夹中随机选取30%的图像进行测试
    for test_set in ['test_set1', 'test_set2', 'test_set3']:
        test_subset = datasets.ImageFolder(root=os.path.join(dataset_path, 'test', test_set), transform=transform)
        test_subset_loader = DataLoader(test_subset, batch_size=64, shuffle=True)

        subset_loss, subset_accuracy = test(model, test_subset_loader, device)

        # 求得损失值的softmax作为增强比
        enhancement_ratios = softmax(torch.tensor(subset_loss), dim=0)

        # 对第二个子训练集进行数据增强
        train_dataset2 = datasets.ImageFolder(root=os.path.join(dataset_path, f'train/{test_set}'), transform=transform)
        enhanced_train_dataset2 = data_augmentation(train_dataset2, enhancement_ratios)

        # 更新训练数据集
        train_loader.dataset.data = enhanced_train_dataset2

        # 继续训练
        train(model, train_loader, criterion, optimizer, epoch, device)

# 在整个测试集上进行最终测试
final_test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
final_test_loss, final_test_accuracy = test(model, final_test_loader, device)

print(f'Final Test Accuracy: {final_test_accuracy:.2f}')
'''

Train Epoch: 0 [0/124 (0%)]	Loss: 1.608904

Test set: Average loss: 1.6041, Accuracy: 0.23

Validation Accuracy after Epoch 1: 0.23
Train Epoch: 1 [0/124 (0%)]	Loss: 1.598004

Test set: Average loss: 1.5982, Accuracy: 0.23

Validation Accuracy after Epoch 2: 0.23
Train Epoch: 2 [0/124 (0%)]	Loss: 1.599200

Test set: Average loss: 1.5915, Accuracy: 0.23

Validation Accuracy after Epoch 3: 0.23
Train Epoch: 3 [0/124 (0%)]	Loss: 1.578173

Test set: Average loss: 1.5855, Accuracy: 0.23

Validation Accuracy after Epoch 4: 0.23
Train Epoch: 4 [0/124 (0%)]	Loss: 1.567704

Test set: Average loss: 1.5793, Accuracy: 0.23

Validation Accuracy after Epoch 5: 0.23
Train Epoch: 5 [0/124 (0%)]	Loss: 1.570063

Test set: Average loss: 1.5722, Accuracy: 0.31

Validation Accuracy after Epoch 6: 0.31
Train Epoch: 6 [0/124 (0%)]	Loss: 1.555111

Test set: Average loss: 1.5644, Accuracy: 0.35

Validation Accuracy after Epoch 7: 0.35
Train Epoch: 7 [0/124 (0%)]	Loss: 1.539087

Test set: Average loss: 1.5561, 

"\nfor epoch in range(10):\n    train(model, train_loader, criterion, optimizer, epoch, device)\n\n    # 在测试集每个子文件夹中随机选取30%的图像进行测试\n    for test_set in ['test_set1', 'test_set2', 'test_set3']:\n        test_subset = datasets.ImageFolder(root=os.path.join(dataset_path, 'test', test_set), transform=transform)\n        test_subset_loader = DataLoader(test_subset, batch_size=64, shuffle=True)\n\n        subset_loss, subset_accuracy = test(model, test_subset_loader, device)\n\n        # 求得损失值的softmax作为增强比\n        enhancement_ratios = softmax(torch.tensor(subset_loss), dim=0)\n\n        # 对第二个子训练集进行数据增强\n        train_dataset2 = datasets.ImageFolder(root=os.path.join(dataset_path, f'train/{test_set}'), transform=transform)\n        enhanced_train_dataset2 = data_augmentation(train_dataset2, enhancement_ratios)\n\n        # 更新训练数据集\n        train_loader.dataset.data = enhanced_train_dataset2\n\n        # 继续训练\n        train(model, train_loader, criterion, optimizer, epoch, device)\n\n# 在整个测试

In [64]:
#进行全部数据增强的
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.functional import softmax
import random
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, ConcatDataset

# 设定种子以确保可重复性
torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Reshape input to (batch_size, sequence_length, input_size)
        x = x.view(x.size(0), x.size(3), -1).permute(0, 2, 1)

        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

# 数据增强
def data_augmentation(dataset):
    # 使用 torchvision 提供的数据增强方法
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform1 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
    ])
    transform2 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform3 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip()
    ])
    transform4 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip()
    ])
    transform5 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform6 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform
    )

    combined_dataset = ConcatDataset([dataset, augmented_dataset])  # 将原数据集和增强后的数据集合并

    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform1
    )
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform2
    )
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform3
    )
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform4
    )
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform5
    )
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform6
    )
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    return combined_dataset

# 训练模型
def train(model, train_loader, criterion, optimizer, epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试模型
def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}\n')

    return test_loss, accuracy

# 数据集路径
dataset_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented'

# 定义转换
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor()])

# 创建数据集和数据加载器
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path,'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path,'val'), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'test'), transform=transform)

# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
input_size = 32  # 输入的大小，根据你的数据集调整
hidden_size = 64  # LSTM隐藏层的大小，根据需要调整
num_classes = 5  # 类别数，根据你的数据集调整
model = LSTMModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

test_dataset_path = os.path.join(dataset_path, 'test')
final_test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)
final_test_loader = DataLoader(final_test_dataset, batch_size=64, shuffle=True)
# 训练10个epoch

for epoch in range(200):
    enhanced_train_dataset = data_augmentation(train_dataset)

    print('增强倍数',len(enhanced_train_dataset) / len(train_dataset))
    # 创建新的 DataLoader 对象，使用 enhanced_train_dataset2
    enhanced_train_loader = DataLoader(enhanced_train_dataset, batch_size=64, shuffle=True)

    # 继续训练
    train(model, enhanced_train_loader, criterion, optimizer, epoch, device)

    # 在验证集上进行测试
    val_loss, val_accuracy = test(model, val_loader, device)
    print(f'Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}')

# 在整个测试集上进行最终测试
final_test_loss, final_test_accuracy = test(model, test_loader, device)
print(f'Final Test Accuracy: {final_test_accuracy:.2f}')

增强倍数 8.0
Train Epoch: 0 [0/992 (0%)]	Loss: 1.600679
Train Epoch: 0 [640/992 (62%)]	Loss: 1.577483

Test set: Average loss: 1.5682, Accuracy: 0.27

Validation Accuracy after Epoch 1: 0.27
增强倍数 8.0
Train Epoch: 1 [0/992 (0%)]	Loss: 1.560701
Train Epoch: 1 [640/992 (62%)]	Loss: 1.512272

Test set: Average loss: 1.4396, Accuracy: 0.46

Validation Accuracy after Epoch 2: 0.46
增强倍数 8.0
Train Epoch: 2 [0/992 (0%)]	Loss: 1.385511
Train Epoch: 2 [640/992 (62%)]	Loss: 1.186623

Test set: Average loss: 1.0894, Accuracy: 0.62

Validation Accuracy after Epoch 3: 0.62
增强倍数 8.0
Train Epoch: 3 [0/992 (0%)]	Loss: 1.109602
Train Epoch: 3 [640/992 (62%)]	Loss: 0.836405

Test set: Average loss: 0.8516, Accuracy: 0.54

Validation Accuracy after Epoch 4: 0.54
增强倍数 8.0
Train Epoch: 4 [0/992 (0%)]	Loss: 0.897352
Train Epoch: 4 [640/992 (62%)]	Loss: 0.761587

Test set: Average loss: 0.7744, Accuracy: 0.73

Validation Accuracy after Epoch 5: 0.73
增强倍数 8.0
Train Epoch: 5 [0/992 (0%)]	Loss: 1.008406
Train Epoch: 

In [12]:
#进行注意力增强的全部文件夹的生成
#本代码实现的功能为生成完全进行数据增强的train_dataset2和train_dataset3,需要配合下面独立开出的代码使用。
#下面的代码主要完成的任务是，利用增强比，从这些数据集中抽取图像数据用于训练
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.functional import softmax
import random
import torch.nn.functional as F
import os
import numpy as np


# 设定种子以确保可重复性
torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Reshape input to (batch_size, sequence_length, input_size)
        x = x.view(x.size(0), x.size(3), -1).permute(0, 2, 1)

        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

# 数据增强
def data_augmentation2(dataset):
    save_path = 'train_dataset2'
    # 使用 torchvision 提供的数据增强方法
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform1 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
    ])
    transform2 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform3 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip()
    ])
    transform4 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip()
    ])
    transform5 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform6 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    #1
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform
    )
    # Save dataset
    os.makedirs(save_path, exist_ok=True)
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_1_{i}.jpg'))

    combined_dataset = ConcatDataset([dataset, augmented_dataset])  # 将原数据集和增强后的数据集合并
    #2
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform1
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_2_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #3
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform2
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_3_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #4
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform3
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_4_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #5
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform4
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_5_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #6
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform5
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_6_{i}.jpg'))

    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #7
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform6
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_7_{i}.jpg'))

    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    return combined_dataset
def data_augmentation3(dataset):
    save_path = 'train_dataset3'
    # 使用 torchvision 提供的数据增强方法
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform1 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
    ])
    transform2 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform3 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip()
    ])
    transform4 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip()
    ])
    transform5 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    transform6 = transforms.Compose([
        transforms.Resize((32, 32)),  # 调整为相同的大小
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ])
    #1
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform
    )
    # Save dataset
    os.makedirs(save_path, exist_ok=True)
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_1_{i}.jpg'))

    combined_dataset = ConcatDataset([dataset, augmented_dataset])  # 将原数据集和增强后的数据集合并
    #2
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform1
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_2_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #3
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform2
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_3_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #4
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform3
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_4_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #5
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform4
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_5_{i}.jpg'))
    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #6
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform5
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_6_{i}.jpg'))

    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])
    #7
    augmented_dataset = datasets.ImageFolder(
        root=dataset.root,
        transform=transform6
    )
    # Save dataset
    for i, (image_tensor, label) in enumerate(augmented_dataset):
        class_name = dataset.classes[label]
        class_path = os.path.join(save_path, class_name)
        os.makedirs(class_path, exist_ok=True)
        image = transforms.ToPILImage()(image_tensor)
        image.save(os.path.join(class_path, f'image_7_{i}.jpg'))

    combined_dataset = ConcatDataset([combined_dataset, augmented_dataset])

    return combined_dataset

# 训练模型
def train(model, train_loader, criterion, optimizer, epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试模型
def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}\n')

    return test_loss, accuracy

def extract_samples_from_concat_dataset(concat_dataset, sampling_ratios):
    num_classes = len(concat_dataset.datasets)
    print("num_classes:",num_classes)
    print('len(concat_dataset):',len(concat_dataset))
    # 定义每个类别的抽取数量
    num_samples_per_class = (sampling_ratios/ sampling_ratios.max() * len(concat_dataset)).int()

    # 创建新的 Subset 数据集
    new_datasets = []
    for class_idx in range(num_classes):
        class_dataset = concat_dataset.datasets[class_idx]
        indices = torch.tensor([i for i, (_, label) in enumerate(class_dataset) if label == class_idx])
        num_samples = min(num_samples_per_class[class_idx], len(indices))

        if num_samples > 0:
            selected_indices = torch.randint(0, len(indices), size=(num_samples,))
            selected_indices = indices[selected_indices]
            new_dataset = Subset(class_dataset, selected_indices)
            new_datasets.append(new_dataset)

    # 创建新的 ConcatDataset
    new_concat_dataset = ConcatDataset(new_datasets)

    return new_concat_dataset

# 数据集路径
dataset_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented1'

# 创建数据集和数据加载器
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor()])
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set1'), transform=transform)
train_dataset2 = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set2'), transform=transform)
#print(train_dataset2)
#print(len(train_dataset2.classes))
train_dataset3 = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set3'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'test'), transform=transform)

# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
input_size = 32  # 输入的大小，根据你的数据集调整
hidden_size = 64  # LSTM隐藏层的大小，根据需要调整
num_classes = 5  # 类别数，根据你的数据集调整
model = LSTMModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练10个epoch
for epoch in range(30):
    # 对第一个拆分后的子训练集进行训练
    train(model, train_loader, criterion, optimizer, epoch, device)

    # 在验证集上进行测试
    val_loss, val_accuracy = test(model, val_loader, device)
    print(f'Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}')

    # 获取每个类别的损失值
    class_losses = [0] * num_classes
    class_counts = [0] * num_classes
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction='none')
            for i in range(num_classes):
                mask = target == i
                class_losses[i] += loss[mask].sum().item()
                class_counts[i] += mask.sum().item()

    # 求得损失值的softmax作为增强比
    enhancement_ratios = softmax(torch.tensor(class_losses) / torch.tensor(class_counts), dim=0)
    print('enhancement_ratios1=',enhancement_ratios)
    # 对第二个划分的子训练集进行数据增强
    enhanced_train_dataset2 = data_augmentation2(train_dataset2)

    enhanced_train_dataset22 = extract_samples_from_concat_dataset(enhanced_train_dataset2, enhancement_ratios)

    # 创建新的 DataLoader 对象，使用 enhanced_train_dataset2
    enhanced_train_loader2 = DataLoader(enhanced_train_dataset22, batch_size=64, shuffle=True)
    '''
    num_samples_per_class = (enhancement_ratios * len(enhanced_train_dataset2)).int()

    # 创建新的 Subset 数据集
    new_datasets = []
    for class_idx in range(5):
        # 遍历每个子数据集
        for dataset_idx, folder_dataset in enumerate(enhanced_train_dataset2.datasets):
            # 计算该子数据集对应的类别的样本数量
            num_samples = int(num_samples_per_class[class_idx] / len(enhanced_train_dataset2))  # 平均分配到每个子数据集

            # 随机选择相应数量的样本
            indices = torch.randperm(len(folder_dataset))
            selected_indices = indices[:num_samples]

            # 创建新的 Subset 数据集
            new_dataset = Subset(folder_dataset, selected_indices)
            new_datasets.append(new_dataset)

    # 创建新的 ConcatDataset
    enhanced_train_dataset22 = ConcatDataset(new_datasets)





    enhanced_train_dataset22 = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set2'), transform=transform)

    for i, (root, dirs, files) in enumerate(os.walk(os.path.join(dataset_path, 'train', 'train_set2'))):
        if i == 0:  # 跳过主目录
            continue
        class_name = os.path.basename(root)
        class_label = train_dataset2.class_to_idx[class_name]
        class_count = int(enhancement_ratios[class_label].item() * len(files))
        selected_files = random.sample(files, class_count)

        for file in selected_files:
            file_path = os.path.join(root, file)
            enhanced_train_dataset22.samples.append((file_path, class_label))

    # 创建新的 DataLoader 对象，使用 enhanced_train_dataset22
    enhanced_train_loader22 = DataLoader(enhanced_train_dataset22, batch_size=64, shuffle=True)

    # 继续训练
    train(model, enhanced_train_loader22, criterion, optimizer, epoch, device)
    '''
    # 继续训练
    train(model, enhanced_train_loader2, criterion, optimizer, epoch, device)

    # 在验证集上进行测试
    val_loss, val_accuracy = test(model, val_loader, device)
    print(f'Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}')

    # 获取每个类别的损失值
    class_losses = [0] * num_classes
    class_counts = [0] * num_classes
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction='none')
            for i in range(num_classes):
                mask = target == i
                class_losses[i] += loss[mask].sum().item()
                class_counts[i] += mask.sum().item()

    # 求得损失值的softmax作为增强比
    enhancement_ratios = softmax(torch.tensor(class_losses) / torch.tensor(class_counts), dim=0)
    print('enhancement_ratios2=',enhancement_ratios)
    # 对第二个划分的子训练集进行数据增强
    enhanced_train_dataset3 = data_augmentation3(train_dataset3)

    enhanced_train_dataset33 = extract_samples_from_concat_dataset(enhanced_train_dataset3, enhancement_ratios)
    # 创建新的 DataLoader 对象，使用 enhanced_train_dataset2
    enhanced_train_loader3 = DataLoader(enhanced_train_dataset33, batch_size=64, shuffle=True)

    # 继续训练
    train(model, enhanced_train_loader3, criterion, optimizer, epoch, device)

# 在整个测试集上进行最终测试
final_test_loss, final_test_accuracy = test(model, test_loader, device)
print(f'Final Test Accuracy: {final_test_accuracy:.2f}')


Train Epoch: 0 [0/42 (0%)]	Loss: 1.608069

Test set: Average loss: 1.6127, Accuracy: 0.21

Validation Accuracy after Epoch 1: 0.21
enhancement_ratios1= tensor([0.2208, 0.1756, 0.2014, 0.2098, 0.1923])
num_classes: 2
len(concat_dataset): 336
Train Epoch: 0 [0/78 (0%)]	Loss: 1.681096

Test set: Average loss: 1.6179, Accuracy: 0.21

Validation Accuracy after Epoch 1: 0.21
enhancement_ratios2= tensor([0.2088, 0.1732, 0.2061, 0.2205, 0.1914])
num_classes: 2
len(concat_dataset): 320
Train Epoch: 0 [0/71 (0%)]	Loss: 1.632756
Train Epoch: 1 [0/42 (0%)]	Loss: 1.601045

Test set: Average loss: 1.6303, Accuracy: 0.21

Validation Accuracy after Epoch 2: 0.21
enhancement_ratios1= tensor([0.1890, 0.1680, 0.2134, 0.2389, 0.1906])
num_classes: 2
len(concat_dataset): 336
Train Epoch: 1 [0/78 (0%)]	Loss: 1.562863

Test set: Average loss: 1.6422, Accuracy: 0.14

Validation Accuracy after Epoch 2: 0.14
enhancement_ratios2= tensor([0.1750, 0.1644, 0.2185, 0.2521, 0.1899])
num_classes: 2
len(concat_dataset)

KeyboardInterrupt: ignored

In [11]:
#本代码用于删除固定路径的文件
import shutil
save_path = 'train_dataset2'
shutil.rmtree(save_path)

In [51]:
#完成基于注意力机制的数据增强的训练
#根据增强比从这些数据集中抽取图像数据用于训练
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.functional import softmax
import random
import torch.nn.functional as F
import os
import numpy as np


# 设定种子以确保可重复性
torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Reshape input to (batch_size, sequence_length, input_size)
        x = x.view(x.size(0), x.size(3), -1).permute(0, 2, 1)

        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

# 训练模型
def train(model, train_loader, criterion, optimizer, epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试模型
def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}\n')

    return test_loss, accuracy
def count_images_per_class(dataset_path):
    class_folders = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]

    image_counts = []

    for class_folder in class_folders:
        class_path = os.path.join(dataset_path, class_folder, class_folder)
        images = [file for file in os.listdir(class_path) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        image_counts.append(len(images))

    return image_counts

def extract_samples_from_concat_dataset(concat_dataset, dataset_path, sampling_ratios):
    num_classes = len(concat_dataset.classes)
    print("num_classes:", num_classes)
    print('len(concat_dataset):', len(concat_dataset))
    print(count_images_per_class(dataset_path))
    #
    # Define the number of samples to extract per class
    num_samples_per_class = (sampling_ratios / sampling_ratios.max() * torch.tensor(count_images_per_class(dataset_path))).int()
    print(num_samples_per_class)

    # Create new Subset datasets
    new_datasets = []
    for class_idx in range(num_classes):
        print('class_idx=',class_idx)  #输出后发现class_idx的输出为0、1、2、3、4,而下方的label应该也和其相对应
        transform = transforms.Compose([
                                transforms.ToTensor()])
        class_path = concat_dataset.classes[class_idx]
        class_folder = os.path.join(concat_dataset.root, class_path)
        print(class_folder)
        class_dataset = datasets.ImageFolder(class_folder, transform=transform)
        #class_dataset = datasets.ImageFolder(concat_dataset.root + '/' + class_path)
        print('class_dataset.label',class_dataset.label)
        indices = torch.tensor([i for i, (_, label) in enumerate(class_dataset) if label == class_idx])
        print(indices)
        num_samples = min(num_samples_per_class[class_idx], len(indices))
        print('num_samples=',num_samples)
        print('len(indices)=',len(indices))

        if num_samples > 0:
            selected_indices = torch.randint(0, len(indices), size=(num_samples,))
            selected_indices = indices[selected_indices]
            new_dataset = Subset(class_dataset, selected_indices)
            new_datasets.append(new_dataset)

    # Create new ConcatDataset
    new_concat_dataset = ConcatDataset(new_datasets)

    return new_concat_dataset
# 数据集路径
dataset_path = '/content/drive/MyDrive/城市工程系统智能化/attention_augmented1'

# 创建数据集和数据加载器
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor()])
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set1'), transform=transform)
#train_dataset2 = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set2'), transform=transform)
train_dataset22 = datasets.ImageFolder(root=os.path.join('/content/drive/MyDrive/城市工程系统智能化/train_dataset2'), transform=transform)
#train_dataset3 = datasets.ImageFolder(root=os.path.join(dataset_path, 'train', 'train_set3'), transform=transform)
train_dataset33 = datasets.ImageFolder(root=os.path.join('/content/drive/MyDrive/城市工程系统智能化/train_dataset3'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'test'), transform=transform)

# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
input_size = 32  # 输入的大小，根据你的数据集调整
hidden_size = 64  # LSTM隐藏层的大小，根据需要调整
num_classes = 5  # 类别数，根据你的数据集调整
model = LSTMModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练10个epoch
for epoch in range(200):
    # 对第一个拆分后的子训练集进行训练
    train(model, train_loader, criterion, optimizer, epoch, device)

    # 在验证集上进行测试
    val_loss, val_accuracy = test(model, val_loader, device)
    print(f'Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}')

    # 获取每个类别的损失值
    class_losses = [0] * num_classes
    class_counts = [0] * num_classes
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction='none')
            for i in range(num_classes):
                mask = target == i
                class_losses[i] += loss[mask].sum().item()
                class_counts[i] += mask.sum().item()

    # 求得损失值的softmax作为增强比
    enhancement_ratios = softmax(torch.tensor(class_losses) / torch.tensor(class_counts), dim=0)
    print('enhancement_ratios1=',enhancement_ratios/ enhancement_ratios.max())
    # 对第二个划分的子训练集进行数据增强
    enhanced_train_dataset22 = extract_samples_from_concat_dataset(train_dataset22, '/content/drive/MyDrive/城市工程系统智能化/train_dataset2',enhancement_ratios)

    # 创建新的 DataLoader 对象，使用 enhanced_train_dataset2
    enhanced_train_loader2 = DataLoader(enhanced_train_dataset22, batch_size=64, shuffle=True)

    # 继续训练
    train(model, enhanced_train_loader2, criterion, optimizer, epoch, device)

    # 在验证集上进行测试
    val_loss, val_accuracy = test(model, val_loader, device)
    print(f'Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}')

    # 获取每个类别的损失值
    class_losses = [0] * num_classes
    class_counts = [0] * num_classes
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction='none')
            for i in range(num_classes):
                mask = target == i
                class_losses[i] += loss[mask].sum().item()
                class_counts[i] += mask.sum().item()

    # 求得损失值的softmax作为增强比
    enhancement_ratios = softmax(torch.tensor(class_losses) / torch.tensor(class_counts), dim=0)
    print('enhancement_ratios2=',enhancement_ratios/ enhancement_ratios.max())
    # 对第二个划分的子训练集进行数据增强
    enhanced_train_dataset33 = extract_samples_from_concat_dataset(train_dataset33,'/content/drive/MyDrive/城市工程系统智能化/train_dataset3', enhancement_ratios)
    # 创建新的 DataLoader 对象，使用 enhanced_train_dataset2
    enhanced_train_loader3 = DataLoader(enhanced_train_dataset33, batch_size=64, shuffle=True)

    # 继续训练
    train(model, enhanced_train_loader3, criterion, optimizer, epoch, device)

# 在整个测试集上进行最终测试
final_test_loss, final_test_accuracy = test(model, test_loader, device)
print(f'Final Test Accuracy: {final_test_accuracy:.2f}')


Train Epoch: 0 [0/42 (0%)]	Loss: 1.608069

Test set: Average loss: 1.6127, Accuracy: 0.21

Validation Accuracy after Epoch 1: 0.21
enhancement_ratios1= tensor([1.0000, 0.7953, 0.9120, 0.9503, 0.8711])
num_classes: 5
len(concat_dataset): 294
[56, 56, 56, 70, 56]
tensor([56, 44, 51, 66, 48], dtype=torch.int32)
class_idx= 0
/content/drive/MyDrive/城市工程系统智能化/train_dataset2/black soil


AttributeError: 'enumerate' object has no attribute 'label'